In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

pd.read_csv('/content/drive/MyDrive/BCICIV_2a_all_patients.csv')

Mounted at /content/drive


,patient,time,label,epoch,EEG-Fz,EEG-0,EEG-1,EEG-2,EEG-3,EEG-4,...,EEG-8,EEG-9,EEG-10,EEG-11,EEG-12,EEG-13,EEG-14,EEG-Pz,EEG-15,EEG-16
0,1,-0.100,tongue,8,-1.681412,2.245496,-0.158350,1.163765,-1.523659,-0.575267,...,0.758116,3.441785,0.305517,1.137473,-1.275763,-2.898359,0.656704,-2.010063,-1.613804,-1.942455
1,1,-0.096,tongue,8,0.420417,0.587559,1.650510,0.970672,1.505904,0.891796,...,1.541586,-0.071620,0.258909,-1.448198,0.142472,-1.968405,-1.733655,-2.935578,-3.125256,-4.674610
2,1,-0.092,tongue,8,0.551365,1.499758,0.121302,2.859433,2.613414,4.636026,...,2.649097,-2.137938,-1.612096,-1.610218,-0.410173,-0.274957,-4.776535,-5.099551,-2.798995,-5.862021
3,1,-0.088,tongue,8,3.054916,-1.807238,1.843603,2.286812,5.995872,6.651295,...,6.031554,-5.249621,-2.672998,-3.452370,0.189081,1.593829,-6.081577,-5.476860,-2.932163,-6.874095
4,1,-0.084,tongue,8,2.506710,-2.453101,0.221178,0.127278,4.519931,6.249573,...,7.827097,-5.309546,-2.488783,-3.707608,1.447515,4.268278,-4.383690,-4.218426,-1.331932,-5.322692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492043,9,0.684,foot,638,7.375522,4.646781,3.071135,4.665561,1.087962,0.932088,...,-2.314982,2.922773,-3.176987,-4.865313,-1.569414,3.600733,-1.113059,-4.249327,-1.402272,-6.307621
492044,9,0.688,foot,638,7.268988,1.952357,5.503664,3.484809,3.227522,-2.738899,...,-2.665657,4.085770,-0.500318,-5.655441,-1.871261,2.517636,-1.805531,-4.453518,-1.801775,-5.974702
492045,9,0.692,foot,638,4.581222,3.463809,4.427226,4.752120,1.906943,-2.496978,...,-5.206939,6.915582,-1.088475,-4.534613,-3.094184,1.148229,-2.247203,-4.895190,-1.803994,-5.879265
492046,9,0.696,foot,638,3.504784,4.096355,2.960162,3.187401,2.148864,-3.182791,...,-5.306815,6.620394,-1.286006,-5.806364,-2.705778,2.269057,-0.540439,-3.530222,-0.194886,-1.486953


In [ ]:
# importing and preprocessing

import numpy as np
import pandas as pd
import torch
import scipy
from torch.utils.data import TensorDataset as TData
from torch.utils.data import DataLoader as DL
import matplotlib.pyplot as plt

# checks if cuda is available, then sets device to GPU
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
device

#read in the data set

dataset = pd.read_csv('/content/drive/MyDrive/BCICIV_2a_all_patients.csv')

# list of columns
print(dataset.columns, end='\n\n') # 22 eeg channels

# unique labels
print(dataset['label'].unique()) # 4 classes

# number of patients
print(dataset.patient.unique())

dataset.describe()

print(dataset.shape)

Index(['patient', 'time', 'label', 'epoch', 'EEG-Fz', 'EEG-0', 'EEG-1',
       'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-Cz',
       'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 'EEG-10', 'EEG-11', 'EEG-12',
       'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16'],
      dtype='object')

['tongue' 'foot' 'right' 'left']
[1 2 3 4 5 6 7 8 9]
(492048, 26)


In [ ]:
# only interested in rows where target is left or right hand in this tutorial for simplicity
binary_mi = dataset.loc[(dataset['label'] == 'left') | (dataset['label'] == 'right')]
binary_mi.label.unique() # check that our row filter worked

array(['right', 'left'], dtype=object)

In [ ]:
# get time period and check that it is constant throughout dataframe
t0 = binary_mi.time.iloc[:-1] # get time points except last one
t1 = binary_mi.time.iloc[1:] # series offest by one, so starting at 2nd time point until last
t0.reset_index(drop=True, inplace=True)
t1.reset_index(drop=True, inplace=True)

print(t0 - t1) # subtracting the two series to get time differences between adjacent time points
const_diff = ((t0 - t1 < -0.0039) & (t0 - t1 > -0.0041)) # checking that they are all around -0.004
const_diff[const_diff != True] # getting just the rows where time difference wasn't around -0.004

0        -0.004
1        -0.004
2        -0.004
3        -0.004
4        -0.004
          ...  
260490   -0.004
260491   -0.004
260492   -0.004
260493   -0.004
260494   -0.004
Name: time, Length: 260495, dtype: float64


,time
200,False
401,False
602,False
803,False
1004,False
...,...
259490,False
259691,False
259892,False
260093,False


In [ ]:
print((t0 - t1)[200])
binary_mi.iloc[200:, 2]# time discrepancy seems to be caused by label change

0.7999999999999999


,label
602,right
603,left
604,left
605,left
606,left
...,...
491842,right
491843,right
491844,right
491845,right


In [ ]:
# don't care about time and epoch anymore
pruned_mi = binary_mi.drop(columns = ['time', 'epoch'])
pruned_mi.head()

,patient,label,EEG-Fz,EEG-0,EEG-1,EEG-2,EEG-3,EEG-4,EEG-5,EEG-C3,...,EEG-8,EEG-9,EEG-10,EEG-11,EEG-12,EEG-13,EEG-14,EEG-Pz,EEG-15,EEG-16
402,1,right,-7.253367,-7.846816,-6.412021,-3.084196,0.023902,1.723496,-4.727450,-4.680500,...,3.113220,-0.901954,1.939467,2.611793,3.706669,3.466285,0.878394,2.311312,2.688791,2.039001
403,1,right,-5.988274,-7.314146,-3.828569,-3.821057,0.312432,-0.771177,-2.241655,-3.757205,...,3.987687,-0.027487,3.302215,1.923760,2.139730,2.143486,1.215752,1.867420,2.293727,1.497453
404,1,right,-5.113807,-3.558819,-2.026367,-3.874324,-1.010367,-2.435772,1.904297,-0.587816,...,4.032076,2.311824,3.004808,-0.277945,0.523963,-0.155874,1.504282,1.716496,0.873272,2.274264
405,1,right,-4.805302,-1.199533,-1.473722,-3.516991,-1.873737,-2.078439,3.970614,2.943346,...,1.069097,3.645720,3.508625,0.274701,-1.169485,-2.972369,2.935834,1.146095,0.058730,2.533941
406,1,right,-4.308143,-0.604718,-0.585937,-3.654597,-2.353140,-4.169171,6.372070,4.417067,...,-1.754056,4.826472,5.031175,-0.155874,-1.648888,-4.672476,3.530649,2.278020,-0.274189,1.468600


In [ ]:
# get unique patients and labels
patients = binary_mi.patient.unique() # 1-9
labels = binary_mi.label.unique()     # left and right

In [ ]:
left_hand = []
right_hand = []

# for each patient, get patient readings, convert to numpy array, and add
# to list corresponding to target (left or right)
for patient in patients:
  left_df = pruned_mi[(pruned_mi['patient'] == patient) & (pruned_mi['label'] == 'left')]
  right_df = pruned_mi[(pruned_mi['patient'] == patient) & (pruned_mi['label'] == 'right')]

  left_hand.append(left_df.iloc[:, 2:].to_numpy().T)
  right_hand.append(right_df.iloc[:, 2:].to_numpy().T)

right_hand[0].shape # shows shape of each list element

(22, 14472)

Getting Lengths and visualizing signals


In [ ]:
# get sequence length/length of each signal
left_len = left_hand[0].shape[1]
right_len = right_hand[0].shape[1]
# portion of sequence used for validation/testing set
left_eval_len = int(0.4 * left_hand[0].shape[1])
right_eval_len = int(0.4 * right_hand[0].shape[1])

# select random starting indexes for sub-sequences for validation and testing sets
left_hand_partition = np.random.randint(0, left_len - left_eval_len, len(left_hand))
right_hand_partition = np.random.randint(0, right_len - right_eval_len, len(right_hand))

# initialize lists to hold respective signals
left_train = []
left_val = []
left_test = []
right_train = []
right_val = []
right_test = []

# create training, validation, and testing sets for left hand signals
for i, num in enumerate(left_hand_partition):
  # get ending index of subsequence being cropped for validation and testing sets
  upper_end = num + left_eval_len
  # add portion of sequence excluding num:upper_end (portion used for evaluation)
  left_train.append(left_hand[i][:, list(range(num)) + list(range(upper_end, left_len))])
  # first half of subsequence used for validation
  left_val.append(left_hand[i][:, num:num+(left_eval_len//2)])
  # second half used for test set
  left_test.append(left_hand[i][:, num+(left_eval_len//2):num + left_eval_len])

# create training, validation, and testing sets for right hand signals
for i, num in enumerate(right_hand_partition):
  # get ending index of subsequence being cropped for validation and testing sets
  upper_end = num + right_eval_len
  # add portion of sequence excluding num:upper_end (portion used for evaluation)
  right_train.append(right_hand[i][:, list(range(num)) + list(range(upper_end, right_len))])
  # first half of subsequence used for validation
  right_val.append(right_hand[i][:, num:num+(right_eval_len//2)])
  # second half used for test set
  right_test.append(right_hand[i][:, num+(right_eval_len//2):num + right_eval_len])

Signal Processing


In [ ]:
# function to bandpass filter signals
def bandpass_filter(signal, crit_freq = [5, 40], sampling_freq = 125):
  order = 4
  b, a = scipy.signal.butter(order, crit_freq, btype = 'bandpass', fs = sampling_freq)
  processed_signal = scipy.signal.filtfilt(b, a, signal, 1)
  return processed_signal

# segment a signal using sliding window technique, specifying sample frequency, window size, and window shift
def segmentation(signal, sampling_freq=125, window_size=1, window_shift=0.016):
  w_size = int(sampling_freq * window_size)
  w_shift = int(sampling_freq * window_shift)
  segments = []
  i = 0
  while i + w_size <= signal.shape[1]:
    segments.append(signal[:, i: i + w_size])
    i += w_shift
  return segments

# apply preprocessing steps in sequence to each signal
def preprocess(signals, crit_freq=[5,35], fs=250):
  preprocessed = []
  for signal in signals:
    # perform bandpass filter on each signal
    filtered_signal = bandpass_filter(signal, crit_freq, fs)
    # channel-wise z-score normalization
    normed_signal = (filtered_signal - filtered_signal.mean(1, keepdims=True)) / filtered_signal.std(1, keepdims=True)
    # segmentation of signals
    segments = segmentation(normed_signal, fs)
    # add signals to list
    preprocessed.extend(segments)
  return preprocessed

In [ ]:
# generate preprocessed segments for training, validation, and testing
train_left = preprocess(left_train)
val_left = preprocess(left_val)
test_left = preprocess(left_test)
train_right = preprocess(right_train)
val_right = preprocess(right_val)
test_right = preprocess(right_test)

# print number of samples corresponding to left and right hand targets
print(f'left samples: {len(train_left) + len(val_left) + len(test_left)}, right samples: {len(train_right) + len(val_right) + len(test_right)}')
# print shape of each sample
print(f'segment shape: {train_left[0].shape}')

left samples: 30897, right samples: 30897
segment shape: (22, 250)


Partitioning Dataset and Creating Data Loaders

In [ ]:
# combine left and right signals to generate signal training, validation, and testing set
train_eeg = train_left + train_right
val_eeg = val_left + val_right
test_eeg = test_left + test_right
train_labels = [0 for i in range(len(train_left))] + [1 for i in range(len(train_right))]
val_labels = [0 for i in range(len(val_left))] + [1 for i in range(len(val_right))]
test_labels = [0 for i in range(len(test_left))] + [1 for i in range(len(test_right))]

In [ ]:
# create torch tensor of zeros to hold data
train_eeg_tensor = torch.zeros((len(train_eeg), train_eeg[0].shape[0], train_eeg[0].shape[1]))
valid_eeg_tensor = torch.zeros((len(val_eeg), val_eeg[0].shape[0], val_eeg[0].shape[1]))
test_eeg_tensor = torch.zeros((len(test_eeg), test_eeg[0].shape[0], test_eeg[0].shape[1]))

# add each sample in train, validation, and test lists to appropriate tensor at correct index
for i in range(len(train_eeg)):
  tens = torch.from_numpy(train_eeg[i].copy())
  train_eeg_tensor[i] = tens
for i in range(len(val_eeg)):
  tens = torch.from_numpy(val_eeg[i].copy())
  valid_eeg_tensor[i] = tens
for i in range(len(test_eeg)):
  tens = torch.from_numpy(test_eeg[i].copy())
  test_eeg_tensor[i] = tens

# create zero tensor for one hot encoded labels
train_label_tensor = torch.zeros(len(train_labels), 2)
valid_label_tensor = torch.zeros(len(val_labels), 2)
test_label_tensor = torch.zeros(len(test_labels), 2)

# add labels to tensor at correct index
for i in range(len(train_labels)):
  label = train_labels[i]
  train_label_tensor[i][label] = 1
for i in range(len(val_labels)):
  label = val_labels[i]
  valid_label_tensor[i][label] = 1
for i in range(len(test_labels)):
  label = test_labels[i]
  test_label_tensor[i][label] = 1

# convert input, target tensors to Tensor Dataset from torch
train_ds = TData(train_eeg_tensor, train_label_tensor)
valid_ds = TData(valid_eeg_tensor, valid_label_tensor)
test_ds = TData(test_eeg_tensor, test_label_tensor)
# create dataloaders to hold batched data (batch size chosen was 64)
train_dl = DL(train_ds, batch_size=64, shuffle= True, drop_last = True)
valid_dl = DL(valid_ds, batch_size=64, shuffle= True, drop_last = True)
test_dl = DL(test_ds, batch_size=64, shuffle = True, drop_last = True)

In [ ]:
# prompt: Given the data make a TCN model that efficiently processes the data, does signal processing, does partitioning Dataset and Creating Data Loaders
# And separable-conv model and efficiently trains the model and evaluates the mod
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

import pandas as pd

# Hyperparameters
input_size = 25  # Adjust based on your data
output_size = 2 # Adjust based on your data
num_channels = [64, 64] # Number of channels in each TCN layer
kernel_size = 3
dropout = 0.2
learning_rate = 0.001
batch_size = 32
num_epochs = 10



# 3. TCN Model
class TCN(nn.Module):
  def __init__(self, in_channels, n_classes):
    super().__init__()
    self.convblock1 = torch.nn.Sequential(torch.nn.Conv1d(in_channels, in_channels, kernel_size = 3, groups = in_channels), # depthwise convolution
                                           torch.nn.Conv1d(in_channels, 32, kernel_size=1), # pointwise convolution
                                           torch.nn.BatchNorm1d(32),
                                           torch.nn.ReLU())

    # self.convblock2 = torch.nn.Sequential(torch.nn.Conv1d(32, 32, kernel_size=3, stride = 2, groups = 32), # depthwise convolution
    #                                        torch.nn.Conv1d(32, 64, kernel_size=1), # pointwise convolution
    #                                        torch.nn.BatchNorm1d(64),
    #                                        torch.nn.ReLU())
    self.flatten = torch.nn.Flatten()
    self.dropout = torch.nn.Dropout(0.5)
    signal_length = 250
    output_size = 32 * 248  # Output size after convblock1 and before flatten

    # Adjust the input size of the linear layer to match the output size of the convolutional layers
    self.dense = torch.nn.Linear(output_size, n_classes)

  def forward(self, x):
    # pass data through both separable conv block
    y = self.convblock1(x)
    # y = self.convblock2(y)
    # flatten data

    y = self.flatten(y)

    # dropout regularization to help mitigate overfitting
    y = self.dropout(y)
    # pass through final linear layer
    y = self.dense(y)
    return y

convNet = TCN(22, 2).to(device)

##### Model Training

from tqdm.auto import tqdm
# using cross_entropy_loss
criterion = torch.nn.CrossEntropyLoss()
# using SGD optimizer for optimal convergence, even if slower
optimizer = torch.optim.SGD(convNet.parameters(), lr = 0.009, momentum = 0.9, weight_decay=5e-3)
epochs = 8

train_losses = []
val_losses = []
accs = []
max_acc = 0
for i in range(epochs):
  total_train_loss = 0.0
  # set model to training mode
  convNet.train()
  # create progress bar
  pbar = tqdm(total=len(train_dl))
  for j, (sig, labels) in tqdm(enumerate(train_dl)):
    # add signal and label to cuda device
    sig = sig.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    # forward pass

    pred = convNet(sig)
    # loss evaluation
    loss = criterion(pred, labels)
    # backpropagation
    loss.backward()
    optimizer.step()

    total_train_loss += loss.item()
    # update progress bar with running loss
    pbar.set_description(f"Epoch {i + 1}    loss={total_train_loss / (j + 1):0.4f}")
    pbar.update(1)
  pbar.close()
  train_losses.append(total_train_loss / len(train_dl))

  total_val_loss = 0.0
  total_accuracy = 0.0
  # set model to evaluation mode, which changes the behavior
  # of some layers like dropout and batch normalization
  convNet.eval()
  with torch.no_grad():
      # create progress bar
      pbar = tqdm(total=len(valid_dl))
      for j, (sig, labels) in enumerate(valid_dl):
          # send signal and label to cuda device
          sig = sig.to(device)
          labels = labels.to(device)
          # make prediction and evaluate loss
          pred = convNet(sig)
          loss = criterion(pred, labels)

          # perform softmax on output which is usually performed by loss automatically
          # this gives a probability vector
          prob_pred = torch.nn.functional.softmax(pred, -1)
          # compute accuracies by
          # 1. getting index of highest number (highest probability prediction)
          # 2. comparing the tensors element wise which gives boolean array comparing predictions to ground truth
          # 3. converting boolean array into array of 0s and 1s
          # 4. calculating mean with binary array, which gives (correct predictions) / (total predictions)
          acc = (prob_pred.argmax(-1) == labels.argmax(-1)).float().mean()
          total_val_loss += loss.item()
          total_accuracy += acc.item()

          # update progress bar with val loss and accuracy
          pbar.set_description(f"val loss={total_val_loss / (j + 1):.4f}    val acc={total_accuracy / (j + 1):.4f}")
          pbar.update(1)
      print()
      pbar.close()
      val_losses.append(total_val_loss / len(valid_dl))
      accs.append(total_accuracy / len(valid_dl))

  # save model if accuracy is best seen
  if accs[-1] > max_acc:
    # dictionary with model state dict, optimizer state dict, and best accuracy
    checkpoint = {'state_dict': convNet.state_dict(), 'optimizer': optimizer.state_dict(), 'best_acc': accs[-1]}
    # save dictionary to specified file path if it exists or create new one otherwise
    torch.save(checkpoint, 'SepConv2.pth.tar')
    print('Model Saved')
    max_acc = accs[-1]




  0%|          | 0/593 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]


Model Saved


  0%|          | 0/593 [00:00<?, ?it/s]

...

  0%|          | 0/593 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]

  0%|          | 0/593 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [ ]:
# loading most accurate model
ckpt_dict = torch.load('SepConv2.pth.tar', weights_only=True)
convNet.load_state_dict(ckpt_dict['state_dict'])
# setting model to evaluation mode
convNet.eval()
test_accuracy = 0.0
test_loss = 0.0
with torch.no_grad():
      for signal, labels in test_dl:
          # send signal and label to cuda device
          signal = signal.to(device)
          labels = labels.to(device)
          # make prediction and evaluate loss
          pred = convNet(signal)
          loss = criterion(pred, labels)
          # compute probability vector and accuracy
          prob_pred = torch.nn.functional.softmax(pred, -1)
          acc = (prob_pred.argmax(-1) == labels.argmax(-1)).float().mean()
          test_loss += loss.item()
          test_accuracy += acc.item()

# display test set loss and accuracy
print(f'Test set    loss: {test_loss / len(test_dl):.4f}    accuracy: {test_accuracy / len(test_dl):.4f}')

Test set    loss: 0.5592    accuracy: 0.7177
